<a href="https://colab.research.google.com/github/Ali-mohammadi-design/Prompt_Engineering_and_Machine_Learning/blob/main/Prompt_engineering_AWS_Bedrock_Text_embedding_Retrieval_Augmented_Generation_RAG_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: In RAG technique we would embed specific text into the model (instead of the fine tuning). Then we would retrieve that text by search based on similarity and would ask from the LLM to consider this text and answer our question. Thus it could be an affordable version of the fine tuning.

In [2]:
! pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 13.3 MB/s eta 0:00:00


In [3]:
import boto3
s3=boto3.resource('s3')

In [5]:
import os
aws_access_key_id=os.environ['aws_access_key_id']
aws_secret_access_key=os.environ['aws_secret_access_key']

In [7]:
s3.buckets.all()
s3 = boto3.resource('s3',
         aws_access_key_id=aws_access_key_id,
         aws_secret_access_key= aws_secret_access_key)

In [8]:
bedrock_runtime= boto3.client(aws_access_key_id=aws_access_key_id,aws_secret_access_key= aws_secret_access_key, service_name='bedrock-runtime', region_name='us-east-1')

In [13]:
sport_text='Baseball is the best sport and there is a baseball game today 4 pm'
finance_text='the stock market was down today by 500 points'

In [9]:
import json

In [11]:
json_request={'inputText':sport_text}

In [12]:
body=json.dumps(json_request)

In [14]:
response=bedrock_runtime.invoke_model(body=body, modelId='amazon.titan-embed-text-v1')

In [15]:
answer=response.get('body').read()
response_body=json.loads(answer)

In [20]:
#response_body['embedding']

Note: we can make a function to embed texts.

In [21]:
def embed_text(text):
  json_request={'inputText':text}
  body=json.dumps(json_request)
  response=bedrock_runtime.invoke_model(body=body, modelId='amazon.titan-embed-text-v1')
  answer=response.get('body').read()
  response_body=json.loads(answer)
  return response_body['embedding']

In [25]:
#embed_text('hello')

Note: How to search based on similarity in the embedded text

In [29]:
import pandas as pd
data={'name':['sport_text','finance_text'], 'text':[sport_text, finance_text]}
df=pd.DataFrame(data)

In [30]:
df

,name,text
0,sport_text,Baseball is the best sport and there is a base...
1,finance_text,the stock market was down today by 500 points


In [31]:
df['embedding']=df['text'].apply(embed_text)

In [32]:
df

,name,text,embedding
0,sport_text,Baseball is the best sport and there is a base...,"[1.2890625, -0.37304688, -0.51953125, -0.12011..."
1,finance_text,the stock market was down today by 500 points,"[0.4375, 0.8359375, -0.51953125, 0.30078125, 0..."


Note: Unlike Langchain, right now there is no specific function or service available to find the similarity in the embedded text. Thus we have to make it ourself with panda and numpy

Note: in the following function we can get the similarities.


Note np.dot() returns the dot product of two arrays.

Note: linalg includes several tools for working with linear algebra problems, including functions for performing matrix calculations, such as determinants, inverses, eigenvalues, eigenvectors, and the singular value decomposition.

numpy.linalg.norm function is used to get the sum from a row or column of a matrix.

In [39]:
import numpy as np

def cosine_similartiy(vector1,vector2):
  vector1=np.array(vector1)
  vector2=np.array(vector2)
  dot_product=np.dot(vector1,vector2)
  mag1=np.linalg.norm(vector1)
  mag2=np.linalg.norm(vector2)
  return dot_product/(mag1*mag2)


In [40]:
vec1=df['embedding'][0]
vec2=df['embedding'][1]

In [35]:
cosine_similartiy(vec1,vec2)

0.1946615328741849

In [36]:
querry='how did the stock market perform today?'

To find the similarity we would embed this prompt and see which embedded text is more similar to this prompt.

In [38]:
prompt_embedding=embed_text(querry)

Note: What is a Lambda Function in Python?
A lambda function is an anonymous function (i.e., defined without a name) that can take any number of arguments but, unlike normal functions, evaluates and returns only one expression.

Refer to : https://www.dataquest.io/blog/tutorial-lambda-functions-in-python/#:~:text=What%20is%20a%20Lambda%20Function,lambda%20parameters%3A%20expression

In [42]:
df['prompt_similarity']=df['embedding'].apply(lambda vector:cosine_similartiy(vector,prompt_embedding))

In [43]:
df

,name,text,embedding,prompt_similarity
0,sport_text,Baseball is the best sport and there is a base...,"[1.2890625, -0.37304688, -0.51953125, -0.12011...",0.196367
1,finance_text,the stock market was down today by 500 points,"[0.4375, 0.8359375, -0.51953125, 0.30078125, 0...",0.731877


Note: as we can see we have the higher similarity with the finance text.

In [48]:
df.nlargest(1,'prompt_similarity').iloc[0]['text']

'the stock market was down today by 500 points'

In [49]:
most_similar_text=df.nlargest(1,'prompt_similarity').iloc[0]['text']

In [50]:
full_prompt=f'answer this question based on provided context. \n question:{querry}. \n context: \n {most_similar_text}'

In [51]:
print(full_prompt)

answer this question based on provided context. 
 question:how did the stock market perform today?. 
 context: 
 the stock market was down today by 500 points


In [53]:
import json
body=json.dumps({'inputText':full_prompt,'textGenerationConfig':{'temperature':0.7 , 'maxTokenCount':500}})
response=bedrock_runtime.invoke_model(body=body, modelId='amazon.titan-text-express-v1')
answer=response.get('body').read()
response_body=json.loads(answer)
output=response_body['results'][0]['outputText']
print(output)

 .
The stock market was down today by 500 points.
